In [16]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# from einops import rearrange, repeat
# import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
from einops import rearrange
### fucking around
# from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
# from train import getSamples, getUpSamples, getSpacing


In [4]:
name = r'building'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (950/971)

In [6]:
sigmoid_ = nn.Sigmoid()
t = torch.tensor(np.linspace(-10, 10, 1000))
y = sigmoid_(t)
local_trans = 1 - y * (1 - y)
s = torch.cumprod(local_trans, 0)



In [26]:
####
# Choose data here
data_path = r'datasets/training_cumulative/round_plant2.npy'
####
with open(data_path, 'rb') as file:
    training_data_np = np.load(file)
print("Loaded data")

Loaded data


In [33]:
x.shape

(300, 3)

In [35]:
quickVizNumpy(x)

In [55]:
from train import getSamplingPositions, getSpacing

t, _ = getSpacing(1, 100)
centre = torch.tensor([[0,0,0]])
dir = torch.tensor([[0,1,0]])
dist = torch.tensor([2])
pos = getSamplingPositions(centre, dir, dist, t)
pos_np = rearrange(np.array(pos), '1 a b -> a b')

In [56]:
quickVizNumpy(pos_np)